https://traitecoevo.github.io/APCalign/articles/APCalign.html

In [1]:
##install.packages("remotes")
remotes::install_github("traitecoevo/APCalign")


withr   (3.0.0 -> 3.0.1 ) [CRAN]
crayon  (1.5.2 -> 1.5.3 ) [CRAN]
openssl (2.1.1 -> 2.2.0 ) [CRAN]
curl    (5.2.0 -> 5.2.1 ) [CRAN]
stringi (1.8.3 -> 1.8.4 ) [CRAN]
arrow   (NA    -> 16.1.0) [CRAN]
readr   (2.1.4 -> 2.1.5 ) [CRAN]


Installing 7 packages: withr, crayon, openssl, curl, stringi, arrow, readr




The downloaded binary packages are in
	/var/folders/0l/lyqlsmz14tq61kh1j0_tqtg80000gq/T//RtmpVSx28c/downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────────────────────────────
✔  checking for file ‘/private/var/folders/0l/lyqlsmz14tq61kh1j0_tqtg80000gq/T/RtmpVSx28c/remotes142b32ff88f66/traitecoevo-APCalign-dfcafde/DESCRIPTION’
─  preparing ‘APCalign’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘APCalign_1.0.2.tar.gz’
   


In [2]:
library(APCalign)

Download the stable version of the taxonomic resources

In [3]:
stable_resources <- load_taxonomic_resources(stable_or_current_data = "stable")


Downloading...

File downloaded successfully.

File downloaded successfully.



Loading resources into memory...



...done



In [4]:
library(RPostgreSQL)
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)

Loading required package: DBI

Warning message:
“package ‘DBI’ was built under R version 4.3.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr



In [5]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")
  tmp <- data.frame(strsplit(tmp,"="))
  dbinfo <- tmp[2,]
  names(dbinfo) <- tmp[1,]
   rm(tmp)
} else {
  cat("No database information found")
}

In [9]:
drv <- dbDriver("PostgreSQL") ## remember to update .pgpass file
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

In [10]:
qry <- "SELECT * FROM form.quadrat_samples"
field_records <- dbGetQuery(con, qry)


str(field_records)
#Close PostgreSQL connection
dbDisconnect(con)


'data.frame':	19242 obs. of  20 variables:
 $ record_id             : int  23233 23234 23235 23236 23237 23238 23239 23240 23241 23242 ...
 $ visit_id              : chr  "CRC09B7UVH" "CRC09B7UVH" "CRC09B7UVH" "CRC09B7UVH" ...
 $ visit_date            : Date, format: "2021-02-03" "2021-02-03" ...
 $ sample_nr             : int  1 1 1 1 1 1 1 1 1 1 ...
 $ species               : chr  "Goodenia ovata" "Gonocarpus tetragynus" "Eucalyptus blaxlandii" "Eucalyptus viminalis" ...
 $ species_code          : int  3192 3247 4058 4197 6403 5697 3540 5445 1211 3824 ...
 $ species_notes         : chr  NA NA NA NA ...
 $ resprout_organ        : chr  NA NA NA NA ...
 $ seedbank              : chr  "Soil-persistent" "Soil-persistent" "Canopy" "Canopy" ...
 $ adults_unburnt        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ resprouts_live        : num  0 100 0 0 32 0 14 0 0 0 ...
 $ resprouts_died        : num  NA NA NA NA NA NA NA NA NA NA ...
 $ resprouts_kill        : num  NA NA NA NA NA NA NA NA NA NA ...
 $

[1] TRUE

In [12]:
library(dplyr)

field_form_names <- field_recordspackageVersion("APCalign") |> 
  pull(species) |> 
  create_taxonomic_update_lookup(resources = stable_resources)

field_form_names |> 
  print(n = 6)

Checking alignments of 1106 taxa


  -> of these 925 names have a perfect match to a scientific name in the APC. 
      Alignments being sought for remaining names.



# A tibble: 19,242 × 12
  original_name       aligned_name accepted_name suggested_name genus taxon_rank
  <chr>               <chr>        <chr>         <chr>          <chr> <chr>     
1 Goodenia ovata      Goodenia ov… Goodenia ova… Goodenia ovata Good… species   
2 Gonocarpus tetragy… Gonocarpus … Gonocarpus t… Gonocarpus te… Gono… species   
3 Eucalyptus blaxlan… Eucalyptus … Eucalyptus b… Eucalyptus bl… Euca… species   
4 Eucalyptus viminal… Eucalyptus … Eucalyptus v… Eucalyptus vi… Euca… species   
5 Pteridium esculent… Pteridium e… Pteridium es… Pteridium esc… Pter… species   
6 Opercularia aspera  Opercularia… Opercularia … Opercularia a… Oper… species   
# ℹ 19,236 more rows
# ℹ 6 more variables: taxonomic_dataset <chr>, taxonomic_status <chr>,
#   scientific_name <chr>, aligned_reason <chr>, update_reason <chr>,
#   number_of_collapsed_taxa <dbl>


In [22]:
field_form_names |> 
    group_by(taxon_rank,update_reason) |> 
    summarise(records=n(), names = n_distinct(original_name))

`summarise()` has grouped output by 'taxon_rank'. You can override using the `.groups` argument.


taxon_rank,update_reason,records,names
<chr>,<chr>,<int>,<int>
family,NA,7,4
genus,NA,316,55
species,aligned name accepted by APC,15924,897
species,basionym,13,4
species,excluded,1,1
species,misapplied,2,1
species,nomenclatural synonym,707,49
species,orthographic variant,68,6
species,pro parte misapplied,22,3


In [21]:
field_form_names |> filter(update_reason %in% "pro parte misapplied") |> distinct()

original_name,aligned_name,accepted_name,suggested_name,genus,taxon_rank,taxonomic_dataset,taxonomic_status,scientific_name,aligned_reason,update_reason,number_of_collapsed_taxa
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Lastreopsis decomposita,Lastreopsis decomposita,Parapolystichum calanthum,Parapolystichum calanthum [alternative possible names: NA],Parapolystichum,species,APC,accepted,Parapolystichum calanthum (Endl.) J.J.S.Gardner & Nagalingum,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-05),pro parte misapplied,1
Rubus fruticosus,Rubus fruticosus,Rubus anglocandicans,Rubus anglocandicans [alternative possible names: Rubus erythrops (pro parte misapplied) | Rubus laciniatus (pro parte misapplied) | Rubus leucostachys (pro parte misapplied) | Rubus phaeocarpus (pro parte misapplied) | Rubus riddelsdellii (pro parte misapplied) | Rubus rubritinctus (pro parte misapplied) | Rubus ulmifolius var. ulmifolius (pro parte misapplied) | Rubus anglocandicans (misapplied) | Rubus laudatus (misapplied)],Rubus,species,APC,accepted,Rubus anglocandicans A.Newton,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-05),pro parte misapplied,1
Salsola kali,Salsola kali,Salsola australis,Salsola australis [alternative possible names: Salsola australis (misapplied)],Salsola,species,APC,accepted,Salsola australis R.Br.,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-05),pro parte misapplied,1


In [24]:
packageVersion("APCalign")
default_version()

[1] ‘1.0.2’

[1] "2024-07+31"

In [25]:
citation("APCalign")

To cite package ‘APCalign’ in publications use:

  Wenk E, Cornwell W, Fuchs A, Kar F, Monro A, Sauquet H, Stephens R,
  Falster D (2024). “APCalign: an R package workflow and app for
  aligning and updating flora names to the Australian Plant Census.”
  _Australian Journal of Botany_. R package version: 1.0.1,
  <https://www.biorxiv.org/content/10.1101/2024.02.02.578715v1>.

A BibTeX entry for LaTeX users is

  @Article{,
    title = {APCalign: an R package workflow and app for aligning and updating flora names to the Australian Plant Census},
    journal = {Australian Journal of Botany},
    author = {Elizabeth Wenk and Will Cornwell and Ann Fuchs and Fonti Kar and Anna Monro and Herve Sauquet and Ruby Stephens and Daniel Falster},
    year = {2024},
    note = {R package version: 1.0.1},
    url = {https://www.biorxiv.org/content/10.1101/2024.02.02.578715v1},
  }